In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
# from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute

In [2]:
e = []
mu = np.array([1., 2., .5])
S = np.array([[1, 0.5, 0.3], [0.5,1., 0.4], [0.3,0.4,1.]])
n_samples= 500
n1, n2 = 300, 200
for i in range(10000):
    X1 = np.random.multivariate_normal(mu, S, n1)
    X2 = np.random.multivariate_normal(mu, S, n2)
    y1 = .3 + X1[:,0] + 0.5*X1[:,1] + X1[:,2] + np.random.normal(0,0.2,n1)
    y2 = .3 + X2[:,0] + 0.5*X2[:,1] + X2[:,2] + np.random.normal(0,0.2,n2)
#     X1, X2, y1, y2 = train_test_split(X,y.reshape((-1,1)),test_size=0.4)
    Xtrain1, Xtest1, ytrain1, ytest1 = train_test_split(X1,y1.reshape((-1,1)),test_size=0.3)
    Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(X2,y2.reshape((-1,1)),test_size=0.3) 
    ndel1, ndel2 = 1,2
    Xtrain1[:, :ndel1] = np.nan
    Xtest1[:,:ndel1] = np.nan
    Xtrain2[:, ndel2:] = np.nan
    Xtest2[:,ndel2:] = np.nan
    
    Xtrain2[:,1] = Xtrain2[:,1] + np.random.normal(0,0.05, len(Xtrain2[:,1]))
    Xtrain2[:,2] = Xtrain2[:,2] + np.random.normal(0,0.2, len(Xtrain2[:,1]))    
    
    Xtrain_merged = np.vstack((Xtrain1, Xtrain2))
    Xtest = np.vstack((Xtest1, Xtest2))
    ymerged = np.vstack((ytrain1, ytrain2))
    ytest = np.vstack((ytest1, ytest2))
    
    Xtrain1 = Xtrain1[:,ndel1:]
    Xtrain2 = Xtrain2[:,:ndel2]
    Xtest1 = Xtest1[:,ndel1:]
    Xtest2 = Xtest2[:,:ndel2]
    
    model1 = LinearRegression().fit(Xtrain1, ytrain1)
    e1 = np.mean(np.abs(model1.predict(Xtest1 - ytest1)))
    
    model2 = LinearRegression().fit(Xtrain2, ytrain2)
    e2 = np.mean(np.abs(model2.predict(Xtest2 - ytest2)))
    
    imputer = SimpleFill(fill_method="mean")
#     imputer =  SoftImpute(max_iters = 100, verbose = False)
    Xtrain_merged = imputer.fit_transform(Xtrain_merged)
    Xtest = imputer.fit_transform(Xtest)
    model_merged = LinearRegression().fit(Xtrain_merged, ymerged)
    e_merged = np.mean(np.abs(model_merged.predict(Xtest) - ytest))
    
    e.append(np.array([e1, e2, e_merged]))    
np.mean(np.asarray(e), axis = 0), np.var(np.asarray(e), axis = 0), 

(array([3.32561618, 3.04366797, 0.7337274 ]),
 array([0.09251443, 0.13109495, 0.00213263]))